In [ ]:
# vssim_loader.py
# ===== VS Code / Jupyter 両対応：$PEDESTRIAN優先 + 無ければ24行スキップ =====
import argparse
from pathlib import Path
import pandas as pd
from io import StringIO
import sys
import os

# ====== 追加: セッション跨ぎで上書きできるデフォルトパス ======
DEFAULT_PATH = r"C:\Users\yonem\OneDrive - 学校法人立命館\デスクトップ\病院デフォルト\デフォルト (8).pp"

def set_default_path(path: str):
    """
    Jupyter 等から、次のセル以降へ引き継がれるデフォルトパスを設定。
    例:
        import vssim_loader as v
        v.set_default_path(r"C:\data\new.pp")  # 以降のセルで有効
        df, df_sorted, result, total, file_path = v.main()  # 引数なしでも new.pp が使われる
    """
    global DEFAULT_PATH
    DEFAULT_PATH = str(path)

def _effective_default_path():
    """
    優先度: 引数(--path) > 環境変数 VSSIM_DEFAULT_PATH > モジュール変数 DEFAULT_PATH
    （※ 引数は argparse 側で処理するので、ここでは環境変数とモジュール変数のみ）
    """
    return os.environ.get("VSSIM_DEFAULT_PATH", DEFAULT_PATH)

# ===============================================================

def load_table_autodetect(path: str) -> pd.DataFrame:
    """
    VISSIM系の pp / txt / csv などを自動読込。
    - $PEDESTRIAN: セクションがあれば最優先でパース
    - それが無ければ 先頭24行スキップ＋区切り推定で読込
    - エンコーディングは utf-8 → cp932 → utf-8-sig を順に試行
    """
    p = Path(path)
    if not p.exists():
        raise FileNotFoundError(f"ファイルが見つかりません: {p}")

    # --- エンコーディング判定しつつ全文取得 ---
    enc_used, text = None, None
    for enc in ("utf-8", "cp932", "utf-8-sig"):
        try:
            text = p.read_text(encoding=enc)
            enc_used = enc
            break
        except Exception:
            continue
    if text is None:
        raise ValueError(f"ファイルを読み取れません（エンコーディング判定失敗）: {p}")

    lines = text.splitlines()

    # ------------- 1) $PEDESTRIAN: を優先 -------------
    ped_idx = None
    for i, ln in enumerate(lines):
        if ln.strip().upper().startswith("$PEDESTRIAN:"):
            ped_idx = i
            break

    def is_comment_or_blank(ln: str) -> bool:
        s = ln.strip()
        return (not s) or s.startswith("*")

    if ped_idx is not None:
        header_line = lines[ped_idx].strip()
        header_after_colon = header_line.split(":", 1)[1] if ":" in header_line else header_line

        # データ行収集（コメント・空行はスキップ、次のセクション開始で打ち切り）
        data_lines = []
        for ln in lines[ped_idx + 1:]:
            if is_comment_or_blank(ln):
                continue
            if ln.strip().startswith("$"):
                break
            data_lines.append(ln)

        if not data_lines:
            raise ValueError("データ行が見つかりません（$PEDESTRIAN: の後に有効な行がありません）。")

        pseudo_csv = header_after_colon + "\n" + "\n".join(data_lines)
        buf = StringIO(pseudo_csv)
        try:
            df_ = pd.read_csv(buf, sep=";", engine="python", on_bad_lines="error")
            used_sep = ";"
        except Exception:
            buf.seek(0)
            df_ = pd.read_csv(buf, sep=r"[;\t,]+", engine="python", on_bad_lines="skip")
            used_sep = r"[;\t,]+"

        # 列名整形 & 互換
        df_.columns = [c.strip() for c in df_.columns]
        if "PEDESTRIAN_NO" not in df_.columns and "NO" in df_.columns:
            df_ = df_.rename(columns={"NO": "PEDESTRIAN_NO"})

        print(f"検出: encoding={enc_used}, section='$PEDESTRIAN', sep={repr(used_sep)}")
        return df_

    # ------------- 2) フォールバック：旧形式（先頭24行スキップ） -------------
    if len(lines) <= 24:
        raise ValueError("ファイルの行数が24行以下のため、削除後に読み込むデータがありません。")
    clean_lines = lines[24:]

    # 区切り文字を推定（; / \t / , の出現数）
    header_line = clean_lines[0]
    seps = [";", "\t", ","]
    sep = max(seps, key=lambda x: header_line.count(x))
    if header_line.count(sep) == 0:
        sep = r"\s+"

    print(f"[fallback] $PEDESTRIAN: なし -> encoding={enc_used}, skipped_first_lines=24, sep={repr(sep)}")
    buf = StringIO("\n".join(clean_lines))
    try:
        df_ = pd.read_csv(buf, sep=sep, engine="python", on_bad_lines="error")
    except Exception as e:
        print(f"[info] 一回目失敗、フォールバックします: {e}")
        buf.seek(0)
        df_ = pd.read_csv(buf, sep=r"[;\t,]+", engine="python", on_bad_lines="skip")

    df_.columns = [c.strip() for c in df_.columns]
    if "PEDESTRIAN_NO" not in df_.columns and "NO" in df_.columns:
        df_ = df_.rename(columns={"NO": "PEDESTRIAN_NO"})
    return df_


In [ ]:
# ===== フィルタ：MOTIONSTATE が 'Walking on level' の行だけ残す =====
from typing import Sequence
import pandas as pd

def filter_walking_on_level(
    df: pd.DataFrame,
    target: str = "Walking on level",
    state_col_candidates: Sequence[str] = ("MOTIONSTATE", "MOTION_STATE"),
) -> pd.DataFrame:
    """
    DataFrame を MOTIONSTATE (候補: 'MOTIONSTATE' / 'MOTION_STATE') が target の行のみに絞る。
    一致判定は文字列化＆strip の上で完全一致。
    - 必須列が無い場合は KeyError
    - 結果が空の場合は Warning を出しつつ空 DataFrame を返す（後段の処理で自然に失敗/スキップ判定できるように）

    返り値: フィルタ後の DataFrame (copy)
    """
    state_col = next((c for c in state_col_candidates if c in df.columns), None)
    if state_col is None:
        raise KeyError(f"必要な列が見つかりません: {list(state_col_candidates)}")

    mask = df[state_col].astype(str).str.strip() == str(target).strip()
    filtered = df.loc[mask].copy()

    if filtered.empty:
        print(f"[WARN] '{state_col}' == '{target}' に一致する行がありません（行数=0）")

    return filtered


In [ ]:

# ========= 旅行時間算出ユーティリティ =========
def compute_travel_time(df: pd.DataFrame):
    """
    各歩行者の DISTTRAVTOT の最小/最大行から SIMSEC を取り、(max - min) を TRAVEL_TIME として返す。
    返り値: (df_sorted, result, total_travel_time_sum)
    """
    # display があれば使う（Jupyter）、無ければ print にフォールバック
    try:
        from IPython.display import display as _display
    except Exception:
        _display = None

    # 列名の自動判定（互換）
    ped_col_candidates  = ['PEDESTRIAN:NO', 'PEDESTRIAN_NO']
    dist_col_candidates = ['DISTTRAVTOT', 'DIST_TRAV_TOT']
    time_col = 'SIMSEC'

    ped_col  = next((c for c in ped_col_candidates  if c in df.columns), None)
    dist_col = next((c for c in dist_col_candidates if c in df.columns), None)

    missing = []
    if ped_col is None:
        missing.append(f"{ped_col_candidates}")
    if dist_col is None:
        missing.append(f"{dist_col_candidates}")
    if time_col not in df.columns:
        missing.append("['SIMSEC']")
    if missing:
        raise KeyError(f"必要な列が見つかりません: {', '.join(missing)}")

    # 1) 並び替え（歩行者→距離 昇順）
    df_sorted = df.sort_values([ped_col, dist_col], ascending=[True, True]).reset_index(drop=True)

    # 2) 各歩行者グループで距離の最小/最大の行インデックス
    idx_min = df_sorted.groupby(ped_col, observed=True)[dist_col].idxmin()
    idx_max = df_sorted.groupby(ped_col, observed=True)[dist_col].idxmax()

    # 3) 最小/最大行から SIMSEC を取得
    min_rows = (df_sorted
                .loc[idx_min, [ped_col, dist_col, time_col]]
                .rename(columns={dist_col: 'DIST_MIN', time_col: 'SIMSEC_MIN'}))

    max_rows = (df_sorted
                .loc[idx_max, [ped_col, dist_col, time_col]]
                .rename(columns={dist_col: 'DIST_MAX', time_col: 'SIMSEC_MAX'}))

    # 4) マージして TRAVEL_TIME を算出
    result = (min_rows
              .merge(max_rows, on=ped_col, how='inner')
              .assign(TRAVEL_TIME=lambda d: d['SIMSEC_MAX'] - d['SIMSEC_MIN'])
              .sort_values(ped_col)
              .reset_index(drop=True))

    total_travel_time_sum = result['TRAVEL_TIME'].sum()

    # 表示（環境に応じて）
    print("並び替え済みデータ（先頭）:")
    if _display:
        _display(df_sorted.head())
    else:
        print(df_sorted.head().to_string(index=False))

    print("各歩行者の最小/最大行と合計旅行時間:")
    if _display:
        _display(result)
    else:
        print(result.to_string(index=False))

    print("全歩行者の合計（TRAVEL_TIME の総和）:", total_travel_time_sum)

    return df_sorted, result, total_travel_time_sum

# ========= 追加：保存ヘルパー（まとめて保存したいときに呼ぶ） =========
def save_parsed_df(df: pd.DataFrame, file_path: str | Path, outdir: str | Path | None = None) -> Path:
    """
    df を CSV 保存。outdir を指定しなければ、元ファイルと同じ場所に *.parsed.csv を作る。
    """
    src = Path(file_path)
    if outdir is None:
        out_path = src.with_suffix(".parsed.csv")
    else:
        outdir = Path(outdir)
        outdir.mkdir(parents=True, exist_ok=True)
        out_path = outdir / src.with_suffix(".parsed.csv").name

    df.to_csv(out_path, index=False, encoding="utf-8-sig")
    print(f"[保存] {out_path}")
    return out_path

# ========= エントリポイント（値を返す＝保持できるように修正） =========
def main(argv=None):
    """
    VS Code/ターミナル/ Jupyter いずれからでも利用可。
    - 値を返すので、呼び出し側で変数として保持できる
    - 既定では保存しない（必要なら save_parsed_df を呼ぶ）
    返り値: (df, df_sorted, result, total_travel_time_sum, file_path)
    """
    parser = argparse.ArgumentParser(
        description="VISSIM系出力の自動読込（$PEDESTRIAN優先 / 無ければ24行スキップ）"
    )
    parser.add_argument(
        "--path", "-p",
        type=str,
        default=None,
        help="入力ファイルパス（--path 未指定時は 環境変数→モジュール既定を使用）"
    )

    # 未知の引数を無視（Jupyter の --f=... 対策）
    args, _unknown = parser.parse_known_args(args=argv)

    # 優先度: 引数 > 環境変数 > モジュール変数
    if args.path and args.path.strip():
        file_path = args.path
        source = "--path"
    else:
        env_or_default = _effective_default_path()
        file_path = env_or_default
        source = "env:VSSIM_DEFAULT_PATH" if os.environ.get("VSSIM_DEFAULT_PATH") else "module:DEFAULT_PATH"

    print(f"[INFO] using path from {source}: {file_path}")

    # --- 読み込み ---
    try:
        df = load_table_autodetect(file_path)
    except Exception as e:
        print(f"[ERROR] 読み込みに失敗しました: {e}", file=sys.stderr)
        # 保存や後続処理ができるよう、例外を再送出せず None を返す
        return None, None, None, None, file_path

    # --- 簡易サマリ ---
    cols = list(df.columns)
    print("列名(先頭30):", cols[:30])
    print("\n----- 先頭5行 -----")
    try:
        from IPython.display import display as _display
        _display(df.head())
    except Exception:
        print(df.head().to_string(index=False))

    # --- 旅行時間 ---
    try:
        df_sorted, result, total = compute_travel_time(df)
    except Exception as e:
        print(f"[WARN] 旅行時間算出に失敗: {e}", file=sys.stderr)
        return df, None, None, None, file_path

    # ここでは保存しない。呼び出し側で必要なら save_parsed_df を呼ぶ。
    return df, df_sorted, result, total, file_path

if __name__ == "__main__":
    # スクリプトとして実行された場合でも、値は作って返す
    df, df_sorted, result, total, file_path = main()
    # 保存はデフォルトでは行わない。必要なら下のコメントアウトを外す。
    # if df is not None:
    #     save_parsed_df(df, file_path)  # 同じフォルダに *.parsed.csv を出力


In [ ]:
# === セル3：セル1で読み込んだ df / result を使って
#      「AGENT_TYPE × STAROUTDECNO ごとの平均 TRAVEL_TIME」を集計（print抑止フラグ付き） ===
import pandas as pd
import numpy as np

def run_agent_star_table(df: pd.DataFrame, result: pd.DataFrame, verbose: bool = True):
    # ---- 列名判定・検証（df / result はセル1で作成済みを想定）----
    ped_col = next((c for c in ['PEDESTRIAN:NO','PEDESTRIAN_NO']
                    if (c in df.columns) or (c in result.columns)), None)
    if ped_col is None:
        raise KeyError("歩行者ID列が見つかりません（PEDESTRIAN:NO / PEDESTRIAN_NO）。")
    if 'TRAVEL_TIME' not in result.columns:
        raise KeyError("result に必須列がありません: TRAVEL_TIME")
    if 'PEDTYPE' not in df.columns:
        raise KeyError("df に PEDTYPE 列がありません。")
    if 'STAROUTDECNO' not in df.columns:
        raise KeyError("df に STAROUTDECNO 列がありません。")

    # 数値化（安全側）
    df = df.copy()
    df['PEDTYPE'] = pd.to_numeric(df['PEDTYPE'], errors='coerce')
    df['STAROUTDECNO'] = pd.to_numeric(df['STAROUTDECNO'], errors='coerce')
    result = result.copy()
    result['TRAVEL_TIME'] = pd.to_numeric(result['TRAVEL_TIME'], errors='coerce')

    # 代表値：最頻値→無ければ先頭（NaN除外）
    def _mode_or_first(s: pd.Series):
        s = s.dropna()
        if s.empty:
            return np.nan
        m = s.mode(dropna=True)
        return m.iloc[0] if len(m) > 0 else s.iloc[0]

    per_ped_meta = (df.groupby(ped_col, observed=True)
                      .agg(PEDTYPE_RAW=('PEDTYPE', _mode_or_first),
                           STAROUTDECNO=('STAROUTDECNO', _mode_or_first))
                      .reset_index())

    # AGENT_TYPE マッピング
    def _agent_map(x) -> str:
        try:
            xi = int(x)
        except Exception:
            return 'Unknown'
        if xi in (100, 200): return 'Human'
        if xi == 300:        return 'Robot'
        return 'Other'
    per_ped_meta['AGENT_TYPE'] = per_ped_meta['PEDTYPE_RAW'].apply(_agent_map)

    # 結合（歩行者単位のメタを result に付与）
    result_with_meta = result.merge(
        per_ped_meta[[ped_col, 'AGENT_TYPE', 'STAROUTDECNO']],
        on=ped_col, how='left'
    )

    # 集計：AGENT_TYPE × STAROUTDECNO ごとの平均 TRAVEL_TIME
    avg = (result_with_meta
           .groupby(['AGENT_TYPE','STAROUTDECNO'], dropna=False)['TRAVEL_TIME']
           .mean()
           .reset_index())

    # STAR を整数表記に（.0 しかなければ Int64）
    if pd.api.types.is_float_dtype(avg['STAROUTDECNO']):
        frac = (avg['STAROUTDECNO'] % 1).fillna(0)
        if (frac == 0).all():
            avg['STAROUTDECNO'] = avg['STAROUTDECNO'].astype('Int64')

    # 並べ替え
    avg = avg.sort_values(['AGENT_TYPE','STAROUTDECNO']).reset_index(drop=True)

    # 表示（任意）
    if verbose:
        print("【AGENT_TYPE × STAROUTDECNO ごとの平均 TRAVEL_TIME】")
        try:
            from IPython.display import display as _display
            _display(avg)
        except Exception:
            print(avg.to_string(index=False))

    return avg, result_with_meta

# ★ 実行（セル1で作られた df / result をそのまま使用）
avg_travel_by_agent_star, result_with_meta = run_agent_star_table(df, result, verbose=True)


In [ ]:
# === AGENT_TYPE × STAROUTDECNO ごとの平均 TRAVEL_TIME（print抑止フラグ付き） ===
import pandas as pd
import numpy as np

def run_agent_star_table(df: pd.DataFrame, result: pd.DataFrame, verbose: bool = True):
    # ---- 列名判定・検証 ----
    ped_col = next((c for c in ['PEDESTRIAN:NO','PEDESTRIAN_NO'] if c in df.columns or c in result.columns), None)
    if ped_col is None:
        raise KeyError("歩行者ID列が見つかりません（PEDESTRIAN:NO / PEDESTRIAN_NO）。")
    if 'TRAVEL_TIME' not in result.columns:
        raise KeyError("result に必須列がありません: TRAVEL_TIME")
    if 'PEDTYPE' not in df.columns:
        raise KeyError("df に PEDTYPE 列がありません。")
    if 'STAROUTDECNO' not in df.columns:
        raise KeyError("df に STAROUTDECNO 列がありません。")

    # 数値化
    df = df.copy()
    df['PEDTYPE'] = pd.to_numeric(df['PEDTYPE'], errors='coerce')
    df['STAROUTDECNO'] = pd.to_numeric(df['STAROUTDECNO'], errors='coerce')
    result = result.copy()
    result['TRAVEL_TIME'] = pd.to_numeric(result['TRAVEL_TIME'], errors='coerce')

    # 代表値：最頻値→無ければ先頭（NaN除外）
    def _mode_or_first(s: pd.Series):
        s = s.dropna()
        if s.empty:
            return np.nan
        m = s.mode(dropna=True)
        return m.iloc[0] if len(m) > 0 else s.iloc[0]

    per_ped_meta = (df.groupby(ped_col, observed=True)
                      .agg(PEDTYPE_RAW=('PEDTYPE', _mode_or_first),
                           STAROUTDECNO=('STAROUTDECNO', _mode_or_first))
                      .reset_index())

    # AGENT_TYPE マッピング
    def _agent_map(x) -> str:
        try:
            xi = int(x)
        except Exception:
            return 'Unknown'
        if xi in (100, 200): return 'Human'
        if xi == 300:        return 'Robot'
        return 'Other'
    per_ped_meta['AGENT_TYPE'] = per_ped_meta['PEDTYPE_RAW'].apply(_agent_map)

    # 結合
    result_with_meta = result.merge(per_ped_meta[[ped_col, 'AGENT_TYPE', 'STAROUTDECNO']],
                                    on=ped_col, how='left')

    # 集計
    avg = (result_with_meta
           .groupby(['AGENT_TYPE','STAROUTDECNO'], dropna=False)['TRAVEL_TIME']
           .mean()
           .reset_index())

    # STAR を整数表記に（.0 しかなければ Int64）
    if pd.api.types.is_float_dtype(avg['STAROUTDECNO']):
        frac = (avg['STAROUTDECNO'] % 1).fillna(0)
        if (frac == 0).all():
            avg['STAROUTDECNO'] = avg['STAROUTDECNO'].astype('Int64')

    # 並べ替え
    avg = avg.sort_values(['AGENT_TYPE','STAROUTDECNO']).reset_index(drop=True)

    # 表示（任意）
    if verbose:
        print("【AGENT_TYPE × STAROUTDECNO ごとの平均 TRAVEL_TIME】")
        print(avg.to_string(index=False))

    return avg, result_with_meta

# ★ 実行（df と result が既にある前提）
avg_travel_by_agent_star, result_with_meta = run_agent_star_table(df, result, verbose=True)  # ←表示したくなければ False


In [ ]:
# === 速度分散 第一処理（堅牢版・Jupyter/ターミナル両対応） ===
import numpy as np
import pandas as pd
import re

# display が使える環境なら使う、なければ print にフォールバック
try:
    from IPython.display import display as _display
except Exception:
    _display = None
def _show(df, head=20, title=None):
    if title: print(title)
    if _display:
        _display(df.head(head))
    else:
        print(df.head(head).to_string(index=False))

# --- df の存在確認 ---
if "df" not in globals() or not isinstance(df, pd.DataFrame):
    raise RuntimeError("df が見つかりません。先に読み込みセルを実行してください。")

work = df.copy()

# === 0) 列名の正規化と基本整形 ===
# 時刻
time_col = (
    'TIME_SEC' if 'TIME_SEC' in work.columns else
    ('SIMSEC' if 'SIMSEC' in work.columns else None)
)
# 歩行者ID（コロン・アンダースコア両対応。誤って $ が付いた列名にも一応対応）
ped_col_candidates = ['PEDESTRIAN_NO', 'PEDESTRIAN:NO', '$PEDESTRIAN:NO']
ped_col = next((c for c in ped_col_candidates if c in work.columns), None)

# 分散列（表記ゆれ両対応）
var_col = (
    'EXPER_VEL_VAR' if 'EXPER_VEL_VAR' in work.columns else
    ('EXPERVELVAR' if 'EXPERVELVAR' in work.columns else None)
)

need_names = {'time': time_col, 'ped': ped_col, 'var': var_col}
missing = [k for k, v in need_names.items() if v is None or v not in work.columns]
if missing:
    raise ValueError(
        f"必要列が見つかりません: {missing}  "
        f"(TIME_SEC / SIMSEC, PEDESTRIAN_NO / PEDESTRIAN:NO, EXPER_VEL_VAR / EXPERVELVAR を確認)"
    )

# === ここから置き換え：COORD → X/Y/Z を確実に用意 ===
def ensure_xyz(df_in: pd.DataFrame) -> pd.DataFrame:
    df_out = df_in.copy()

    # 既に X/Y/Z があれば数値化して返す
    if {'X','Y','Z'}.issubset(df_out.columns):
        for c in ['X','Y','Z']:
            df_out[c] = pd.to_numeric(df_out[c], errors='coerce')
        return df_out

    # 1) 既知の候補名を優先
    known_names = {'COORD_CENT','COORDCENT','COORD_CENTER','COORDCENTER','COORD'}
    cand = [c for c in df_out.columns if c.upper() in known_names]

    # 2) 見つからなければ object 列を広めに候補化
    if not cand:
        cand = [c for c in df_out.columns if df_out[c].dtype == object]

    # 数値3連（空白/カンマ/セミコロン区切り）検出
    num = r'([+-]?\d+(?:\.\d+)?(?:[eE][+-]?\d+)?)'
    pat = re.compile(r'^\s*' + num + r'[\s,;]+' + num + r'[\s,;]+' + num + r'\s*$')

    best_col, best_match, best_count = None, None, -1
    for c in cand:
        s = df_out[c].astype(str)
        m = s.str.extract(pat)
        cnt = (m.notna().all(axis=1)).sum()
        if cnt > best_count:
            best_col, best_match, best_count = c, m, cnt

    if best_col is not None and best_count > 0:
        best_match.columns = ['X','Y','Z']
        for col in ['X','Y','Z']:
            df_out[col] = pd.to_numeric(best_match[col], errors='coerce')
    else:
        for col in ['X','Y','Z']:
            if col not in df_out.columns:
                df_out[col] = np.nan

    return df_out

work = ensure_xyz(work)
# === 置き換えここまで ===

# 数値化（速度は VISSIM の列名ゆらぎも面倒見る：SPEED / DES_SPEED / DESSPEED）
numeric_candidates = [time_col, var_col, 'X','Y','Z','SPEED','DES_SPEED','DESSPEED']
for c in numeric_candidates:
    if c in work.columns:
        work[c] = pd.to_numeric(work[c], errors='coerce')

# 並び替え（歩行者→時刻）
work.sort_values([ped_col, time_col], inplace=True, kind='mergesort')

# 衝突しない元行番号列名を作成
orig_idx_col = '__ROW_ORIG__'
while orig_idx_col in work.columns:
    orig_idx_col += '_X'

# 元 index を保存（重複回避版）
work.reset_index(drop=False, inplace=True)
work.rename(columns={'index': orig_idx_col}, inplace=True)

# === 1) しきい値（分散 σv^2 ベース） ===
bins_var = [-np.inf, 0.02, 0.25, 1.0, 4.0, 9.0, np.inf]
labels = [
    '等速（ゼロ帯）',     # ≤0.02
    '通常',               # (0.02, 0.25]
    '軽度不安定',         # (0.25, 1.0]
    '中等度不安定',       # (1.0, 4.0]
    '物理限界超え疑い',   # (4.0, 9.0]
    '非現実（異常確定）'  # >9.0
]

work['VAR'] = work[var_col]
work['STD'] = np.sqrt(work['VAR'])
work['EXPERVELVAR_LABEL'] = pd.cut(
    work['VAR'], bins=bins_var, labels=labels, include_lowest=True, right=True
)

# === 2) “同ラベルが連続” を 1現象にまとめる（歩行者ごと） ===
tmp_label = (
    work['EXPERVELVAR_LABEL']
      .astype('object')
      .where(work['EXPERVELVAR_LABEL'].notna(), '（未分類）')
)

work['_label_block_id'] = (
    tmp_label
    .groupby(work[ped_col])                       # Series の groupby
    .apply(lambda s: (s != s.shift()).cumsum())
    .reset_index(level=0, drop=True)
    .astype(int)
)

# === 3) エピソード（連続区間）要約 ===
def trend_label(dv, eps=0.05):
    if pd.isna(dv): return '不明'
    if dv >  eps:   return '速度増加'
    if dv < -eps:   return '速度低下'
    return '概ね横ばい'

def safe_get(series, pos):
    try:
        return series.iloc[pos]
    except Exception:
        return np.nan

episodes_rows = []
for (pid, blk), g in work.groupby([ped_col, '_label_block_id'], sort=True):
    lab = g['EXPERVELVAR_LABEL'].iloc[0]
    lab_str = lab if pd.notna(lab) else '（未分類）'
    t0 = g[time_col].iloc[0]; t1 = g[time_col].iloc[-1]
    n  = len(g)

    # 速度トレンド（列名ゆらぎ対応：DES_SPEED or DESSPEED は参考値）
    speed0 = g['SPEED'].iloc[0] if 'SPEED' in g.columns else np.nan
    speed1 = g['SPEED'].iloc[-1] if 'SPEED' in g.columns else np.nan
    d_speed = (speed1 - speed0) if (pd.notna(speed0) and pd.notna(speed1)) else np.nan
    tr = trend_label(d_speed)

    # 座標
    x0, y0, z0 = safe_get(g['X'],0),  safe_get(g['Y'],0),  safe_get(g['Z'],0)
    x1, y1, z1 = safe_get(g['X'],-1), safe_get(g['Y'],-1), safe_get(g['Z'],-1)
    x_med = g['X'].median() if 'X' in g.columns else np.nan
    y_med = g['Y'].median() if 'Y' in g.columns else np.nan
    z_med = g['Z'].median() if 'Z' in g.columns else np.nan

    # 区間中の分散/標準偏差指標
    var_max = g['VAR'].max()
    var_med = g['VAR'].median()
    std_med = g['STD'].median()

    # 元行位置（nullable int へ）
    row_start_val = g[orig_idx_col].iloc[0]
    row_end_val   = g[orig_idx_col].iloc[-1]
    row_start = pd.to_numeric(pd.Series([row_start_val]), errors='coerce').iloc[0]
    row_end   = pd.to_numeric(pd.Series([row_end_val]),   errors='coerce').iloc[0]

    episodes_rows.append({
        ped_col: pid,
        'BLOCK_ID': int(blk),
        'LABEL': lab_str,
        'N_SAMPLES': n,
        'T_START': t0,
        'T_END': t1,
        'DURATION': (t1 - t0) if (pd.notna(t0) and pd.notna(t1)) else np.nan,
        'VAR_MED': var_med,
        'VAR_MAX': var_max,
        'STD_MED': std_med,
        'SPEED_START': speed0,
        'SPEED_END': speed1,
        'DELTA_SPEED': d_speed,
        'SPEED_TREND': tr,
        'X_START': x0, 'Y_START': y0, 'Z_START': z0,
        'X_END':   x1, 'Y_END':   y1, 'Z_END':   z1,
        'X_MED': x_med, 'Y_MED': y_med, 'Z_MED': z_med,
        'ROW_START': row_start,
        'ROW_END':   row_end,
    })

episodes = (
    pd.DataFrame(episodes_rows)
      .sort_values([ped_col, 'T_START', 'BLOCK_ID'])
      .reset_index(drop=True)
)

# ROW_* は nullable 整数に（NaN を許容）
for c in ['ROW_START','ROW_END']:
    if c in episodes.columns:
        episodes[c] = episodes[c].astype('Int64')

# === 4) ダッシュボード的な集計 ===
abnormal_levels = ['中等度不安定','物理限界超え疑い','非現実（異常確定）']
abnormal = episodes[episodes['LABEL'].isin(abnormal_levels)]

count_table = (
    episodes
    .groupby([ped_col, 'LABEL'], dropna=False)
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# 出力
_show(episodes, head=20, title="=== episodes（先頭）===")
_show(count_table, head=20, title="=== 歩行者×ラベル件数（先頭）===")

# === 5) CSV保存（Excel互換の utf-8-sig） ===
#episodes.to_csv('expvelvar_episodes.csv', index=False, encoding='utf-8-sig')
#count_table.to_csv('expvelvar_counts_by_ped.csv', index=False, encoding='utf-8-sig')
#abnormal.to_csv('expvelvar_abnormal_only.csv', index=False, encoding='utf-8-sig')
#print("保存: expvelvar_episodes.csv / expvelvar_counts_by_ped.csv / expvelvar_abnormal_only.csv")


In [ ]:
# === 速度分散 第二処理（堅牢版・Jupyter/ターミナル両対応・保存しない版） ===
import pandas as pd

try:
    from IPython.display import display as _display
except Exception:
    _display = None

def _show(df, title):
    print(title)
    if _display: _display(df)
    else: print(df.to_string())

def run_expvelvar_second(episodes: pd.DataFrame):
    # 1) 前提チェック
    if 'episodes' not in globals() or not isinstance(episodes, pd.DataFrame):
        raise RuntimeError("先にエピソード抽出セル（第一処理）を実行して、episodes を作ってください。")
    if 'LABEL' not in episodes.columns:
        raise KeyError("episodes に 'LABEL' 列が見つかりません。第一処理の結果を確認してください。")

    # 2) 歩行者ID列の特定
    ped_candidates = ['PEDESTRIAN:NO', 'PEDESTRIAN_NO']
    ped_col = next((c for c in ped_candidates if c in episodes.columns), None)
    if ped_col is None:
        known_non_id = {
            'BLOCK_ID','LABEL','N_SAMPLES','T_START','T_END','DURATION',
            'VAR_MED','VAR_MAX','STD_MED','SPEED_START','SPEED_END','DELTA_SPEED','SPEED_TREND',
            'X_START','Y_START','Z_START','X_END','Y_END','Z_END','X_MED','Y_MED','Z_MED','ROW_START','ROW_END'
        }
        candidates = [c for c in episodes.columns if c not in known_non_id]
        ped_col = candidates[0] if candidates else episodes.columns[0]

    # 3) ラベル順
    base_order = [
        '等速（ゼロ帯）','通常','軽度不安定','中等度不安定','物理限界超え疑い','非現実（異常確定）','（未分類）'
    ]
    existing = episodes['LABEL'].dropna().astype(str).unique().tolist()
    tail = [l for l in existing if l not in base_order]
    level_order = base_order + tail
    labels_cat = pd.Categorical(episodes['LABEL'].astype('object'), categories=level_order, ordered=True)

    # 4) レベル別総件数
    level_counts = (
        pd.Series(labels_cat, name='EXPERVELVAR_LEVEL')
          .value_counts(dropna=False)
          .sort_index()
          .reset_index(name='COUNT')
          .rename(columns={'index': 'EXPERVELVAR_LEVEL'})
    )
    _show(level_counts, "=== EXPERVELVAR レベル別の発生回数 ===")

    # 5) 歩行者 × レベル 件数
    level_by_ped = (
        episodes.assign(_LABEL=labels_cat)
                .groupby([ped_col, '_LABEL'], dropna=False)
                .size()
                .unstack('_LABEL', fill_value=0)
                .reindex(columns=level_order, fill_value=0)
    )
    try: level_by_ped = level_by_ped.sort_index()
    except Exception: pass
    _show(level_by_ped, "=== 歩行者別 × レベル別 件数 ===")

    # ここでは保存しない。呼び出し側で保存。
    return level_counts, level_by_ped

# ★ 実行（episodes がある前提）
level_counts, level_by_ped = run_expvelvar_second(episodes)


In [ ]:
# === 速度分散：総平均 / PEDTYPE 集約 / 時間ビン / STAROUTDECNO 集約（堅牢版） ===
import numpy as np
import pandas as pd

# display が使える環境なら使う、なければ print にフォールバック
try:
    from IPython.display import display as _display
except Exception:
    _display = None
def _show(df, title, head=None):
    print(title)
    if head is not None:
        df = df.head(head)
    if _display:
        _display(df)
    else:
        print(df.to_string(index=False))

# --- df の存在確認 ---
if "df" not in globals() or not isinstance(df, pd.DataFrame):
    raise RuntimeError("df が見つかりません。先に読み込みセルを実行してください。")

w = df.copy()

# --- 必要列の特定（柔軟対応） ---
# 速度分散列
var_col = None
for c in ("EXPER_VEL_VAR", "EXPERVELVAR"):
    if c in w.columns:
        var_col = c
        break

# 時刻列（SIMSEC を優先。無ければ TIME_SEC）
time_col = None
for c in ("SIMSEC", "TIME_SEC"):
    if c in w.columns:
        time_col = c
        break

pedtype_exists   = "PEDTYPE"      in w.columns
starout_exists   = "STAROUTDECNO" in w.columns

need = {
    "速度分散列 (EXPER_VEL_VAR/EXPERVELVAR)": var_col,
    "時刻列 (SIMSEC/TIME_SEC)": time_col,
    "PEDTYPE": "PEDTYPE" if pedtype_exists else None,
    "STAROUTDECNO": "STAROUTDECNO" if starout_exists else None,
}
missing = [k for k, v in need.items() if v is None]
if missing:
    raise ValueError(f"必要列が見つかりません: {missing}")

# --- 数値化（安全） ---
w[var_col] = pd.to_numeric(w[var_col], errors="coerce")
w[time_col] = pd.to_numeric(w[time_col], errors="coerce")
w["PEDTYPE"] = pd.to_numeric(w["PEDTYPE"], errors="coerce")
w["STAROUTDECNO"] = pd.to_numeric(w["STAROUTDECNO"], errors="coerce")

# --- PEDTYPE を 3分類にマッピング（100/200=HUMAN, 300=ROBOT, その他=OTHER） ---
def _agent_group(x) -> str:
    try:
        xi = int(x)
    except Exception:
        return "OTHER"
    if xi in (100, 200): return "HUMAN"
    if xi == 300:        return "ROBOT"
    return "OTHER"

w["AGENT_GROUP"] = w["PEDTYPE"].apply(_agent_group)

# ---------------------------------------------------------------
# 1) 全体の平均速度分散
# ---------------------------------------------------------------
overall_mean = w[var_col].mean(skipna=True)
overall_df = pd.DataFrame({"METRIC": ["OVERALL_MEAN_VAR"], "MEAN_VAR": [overall_mean]})
_show(overall_df, "=== 全体の平均速度分散 ===")

# ---------------------------------------------------------------
# 2) AGENT_GROUP 別（HUMAN / ROBOT / OTHER）の平均速度分散
# ---------------------------------------------------------------
by_type = (
    w.groupby("AGENT_GROUP", dropna=False, as_index=False)[var_col]
     .mean(numeric_only=True)
     .rename(columns={var_col: "MEAN_VAR"})
     .sort_values("AGENT_GROUP")
     .reset_index(drop=True)
)
_show(by_type, "=== AGENT_GROUP 別 平均速度分散（HUMAN/ROBOT/OTHER） ===")

# ---------------------------------------------------------------
# 3) AGENT_GROUP 別 × 時間ビン（既定 1800 秒）の平均速度分散
#    到着基準にしたい場合は time_col を変更してください
# ---------------------------------------------------------------
BIN = 1800  # 30分=1800秒

# 欠損は除外しつつビンを算出
t = pd.to_numeric(w[time_col], errors="coerce")
# NaN を一時的に -1 にして floor_divide、負値はのちに NaN に戻す
bin_start = np.floor_divide(t.fillna(-1).astype("int64"), BIN) * BIN
bin_start = bin_start.where(bin_start >= 0, pd.NA).astype("Int64")

w_time = w.copy()
w_time["TIME_BIN_START"] = bin_start
w_time = w_time[w_time["TIME_BIN_START"].notna()].copy()

def _sec_to_hhmmss(x) -> str:
    if pd.isna(x):
        return "NaT"
    x = int(x)
    h = x // 3600
    m = (x % 3600) // 60
    s = x % 60
    return f"{h:02d}:{m:02d}:{s:02d}"

w_time["TIME_BIN_LABEL"] = w_time["TIME_BIN_START"].apply(_sec_to_hhmmss)

by_type_time = (
    w_time.groupby(["AGENT_GROUP", "TIME_BIN_START", "TIME_BIN_LABEL"], dropna=False, as_index=False)[var_col]
          .mean(numeric_only=True)
          .rename(columns={var_col: "MEAN_VAR"})
          .sort_values(["AGENT_GROUP", "TIME_BIN_START", "TIME_BIN_LABEL"])
          .reset_index(drop=True)
)
_show(by_type_time, f"=== AGENT_GROUP 別 × 時間ビン（{BIN}秒）平均速度分散 ===", head=30)

# ---------------------------------------------------------------
# 4) AGENT_GROUP 別 × STAROUTDECNO 別 平均速度分散
# ---------------------------------------------------------------
by_type_star = (
    w.groupby(["AGENT_GROUP", "STAROUTDECNO"], dropna=False, as_index=False)[var_col]
     .mean(numeric_only=True)
     .rename(columns={var_col: "MEAN_VAR"})
     .sort_values(["AGENT_GROUP", "STAROUTDECNO"])
     .reset_index(drop=True)
)

# STAROUTDECNO が実数型かつ .0 なら Int64 に見せる
if pd.api.types.is_float_dtype(by_type_star["STAROUTDECNO"]):
    frac = (by_type_star["STAROUTDECNO"] % 1).fillna(0)
    if (frac == 0).all():
        by_type_star["STAROUTDECNO"] = by_type_star["STAROUTDECNO"].astype("Int64")

_show(by_type_star, "=== AGENT_GROUP 別 × STAROUTDECNO 別 平均速度分散 ===", head=30)

# ---（必要ならCSV保存：Excel互換の UTF-8 BOM 付き）---
# overall_df.to_csv('mean_var_overall.csv', index=False, encoding='utf-8-sig')
# by_type.to_csv('mean_var_by_type.csv', index=False, encoding='utf-8-sig')
# by_type_time.to_csv('mean_var_by_type_timebin.csv', index=False, encoding='utf-8-sig')
# by_type_star.to_csv('mean_var_by_type_star.csv', index=False, encoding='utf-8-sig')
# print("保存: mean_var_overall.csv / mean_var_by_type.csv / mean_var_by_type_timebin.csv / mean_var_by_type_star.csv")


In [ ]:
# === イベント回数（堅牢版・Jupyter/ターミナル両対応｜保存しない版） ===
import pandas as pd
import numpy as np
import unicodedata as ud
import re

# 表示ユーティリティ
try:
    from IPython.display import display as _display
except Exception:
    _display = None
def _show(df, title=None, head=None):
    if title: print(title)
    if head is not None:
        df = df.head(head)
    if _display: _display(df)
    else: print(df.to_string(index=False))

# --- パラメータ ---
TOL_NEIGHBOR_SEC = 0.5
EVENT_GAP_SEC    = 1.0
CLUSTER_SEC      = 3.0
DIST_LIMIT       = 0.5
PASS_BY_DEG      = 100

# --- df の存在確認 ---
if "df" not in globals() or not isinstance(df, pd.DataFrame):
    raise RuntimeError("df が見つかりません。先に読み込みセルを実行してください。")

# --- ユーティリティ ---
def norm(s: str) -> str:
    if s is None: return ""
    s = ud.normalize("NFKC", str(s)).strip().lower()
    s = s.replace("$","").replace("：",":").replace("（","(").replace("）",")")
    for ch in [" ", "\t", "-", "/", "\\", ":"]:
        s = s.replace(ch, "_")
    while "__" in s: s = s.replace("__","_")
    return s

def find_col(df_: pd.DataFrame, candidates) -> str:
    nmap = {c: norm(c) for c in df_.columns}
    wants = {norm(c) for c in candidates}
    for col, coln in nmap.items():
        if coln in wants:
            return col
    return ""

def to_num(s):
    try:
        return pd.to_numeric(s, errors="coerce")
    except Exception:
        return s

# --- 列名検出（実列名を変数に保持：リネームしない）---
COL_ID   = find_col(df, ["$PEDESTRIAN:NO","PEDESTRIAN:NO","PEDESTRIAN_NO","AGENT_ID","ID","AgentID"])
COL_TYPE = find_col(df, ["PEDTYPE","TYPE"])
COL_ANG  = find_col(df, ["ORIENTATIONANGLE","ORIENTATION_ANGLE","ANGLE"])
COL_T    = find_col(df, ["SIMSEC","TIME","TIMESTAMP","SEC"])
COL_NID  = find_col(df, ["NEARNEIGHB","NEAR_NEIGHB","NEIGHBOR","NEIGHBORID"])
COL_ND   = find_col(df, ["NEARNEIGHBDIST","NEAR_NEIGHB_DIST","NEIGHBORDIST","NEIGHBOR_DIST"])
COL_COORD= find_col(df, ["COORDCENT","COORD","COORD_CENTER","CENTROID"])

COL_X = find_col(df, ["X","XCOORD","XCoord","$PEDESTRIAN:X","POSX","X [m]","X[m]","x","pos_x"])
COL_Y = find_col(df, ["Y","YCOORD","YCoord","$PEDESTRIAN:Y","POSY","Y [m]","Y[m]","y","pos_y"])
COL_Z = find_col(df, ["Z","ZCOORD","ZCoord","$PEDESTRIAN:Z","POSZ","Z [m]","Z[m]","z","pos_z"])

need = [COL_ID, COL_TYPE, COL_ANG, COL_T, COL_NID, COL_ND]
if any(c == "" for c in need):
    names = ["ID","PEDTYPE","ORIENTATIONANGLE","SIMSEC","NEARNEIGHB","NEARNEIGHBDIST"]
    have  = [bool(COL_ID),bool(COL_TYPE),bool(COL_ANG),bool(COL_T),bool(COL_NID),bool(COL_ND)]
    lack  = [n for n, ok in zip(names, have) if not ok]
    raise KeyError(f"必須列が見つかりません: {lack}")

# --- 数値化 ---
for c in [COL_ID, COL_TYPE, COL_ANG, COL_T, COL_NID, COL_ND, COL_X, COL_Y, COL_Z]:
    if c:
        df[c] = to_num(df[c])

# --- COORDCENT → SelfX/Y/Z（X/Yが無い時のみ）---
if (not COL_X or not COL_Y) and COL_COORD:
    pat = re.compile(r"\s*([+-]?\d+(?:\.\d+)?)\s+([+-]?\d+(?:\.\d+)?)\s+([+-]?\d+(?:\.\d+)?)\s*")
    xyz = df[COL_COORD].astype(str).str.extract(pat)
    df["SelfX"] = pd.to_numeric(xyz[0], errors="coerce")
    df["SelfY"] = pd.to_numeric(xyz[1], errors="coerce")
    df["SelfZ"] = pd.to_numeric(xyz[2], errors="coerce")
else:
    df["SelfX"] = pd.to_numeric(df[COL_X], errors="coerce") if COL_X else np.nan
    df["SelfY"] = pd.to_numeric(df[COL_Y], errors="coerce") if COL_Y else np.nan
    df["SelfZ"] = pd.to_numeric(df[COL_Z], errors="coerce") if COL_Z else np.nan

# --- 角度だけ asof で取得（Neighbor座標は扱わない）---
left = df[[COL_ID, COL_T, COL_NID, COL_ND, COL_ANG, COL_TYPE, "SelfX","SelfY","SelfZ"]].copy()
left = left.rename(columns={COL_T:"SIMSEC"})
right = df[[COL_ID, COL_T, COL_ANG]].copy().rename(columns={COL_T:"SIMSEC"})

left["match_id"] = pd.to_numeric(left[COL_NID], errors="coerce")
right["match_id"] = pd.to_numeric(right[COL_ID], errors="coerce")
left["SIMSEC"]   = pd.to_numeric(left["SIMSEC"], errors="coerce")
right["SIMSEC"]  = pd.to_numeric(right["SIMSEC"], errors="coerce")
left["NEARNEIGHBDIST"]   = pd.to_numeric(left[COL_ND], errors="coerce")
left["ORIENTATIONANGLE"] = pd.to_numeric(left[COL_ANG], errors="coerce")
left["PEDTYPE"]          = pd.to_numeric(left[COL_TYPE], errors="coerce")

left_nn  = left[left["match_id"].notna() & left["SIMSEC"].notna()].copy()
left_na  = left[left["match_id"].isna()  | left["SIMSEC"].isna()].copy()
right_nn = right[right["match_id"].notna() & right["SIMSEC"].notna()].copy().rename(
    columns={COL_ANG:"NeighborAngle"}
)

pieces = []
if not left_nn.empty:
    for mid, lgrp in left_nn.groupby("match_id", sort=False):
        rgrp = right_nn[right_nn["match_id"] == mid]
        lsorted = lgrp.sort_values("SIMSEC", kind="mergesort").reset_index(drop=True)
        if rgrp.empty:
            out = lsorted.copy()
            out["NeighborAngle"] = np.nan
        else:
            rsorted = rgrp.sort_values("SIMSEC", kind="mergesort").reset_index(drop=True)
            out = pd.merge_asof(
                lsorted, rsorted[["SIMSEC","NeighborAngle"]],
                on="SIMSEC", direction="nearest", tolerance=TOL_NEIGHBOR_SEC
            )
        pieces.append(out)

merged_nn = pd.concat(pieces, ignore_index=True, sort=False) if pieces else left_nn.copy()
if "NeighborAngle" not in left_na.columns:
    left_na["NeighborAngle"] = np.nan
merged = pd.concat([merged_nn, left_na], ignore_index=True, sort=False)

# ★修正点1：NEARNEIGHB は自分自身の match_id を採用（別DFからの代入を禁止）
merged["NEARNEIGHB"] = merged["match_id"]

# --- ラベル付け・相互作用 ---
id_to_type = (
    df[[COL_ID, COL_TYPE]].dropna().drop_duplicates().set_index(COL_ID)[COL_TYPE].to_dict()
)

def safe_get_type(x):
    try:
        return id_to_type.get(float(x))
    except Exception:
        return np.nan

def label_from_type(p):
    if pd.isna(p): return "Unknown"
    if p in (100, 200): return "Human"
    if p == 300:        return "Robot"
    return "Unknown"

merged["NeighborType"]  = merged["NEARNEIGHB"].map(safe_get_type)
merged["SelfLabel"]     = merged["PEDTYPE"].map(label_from_type)
merged["NeighborLabel"] = merged["NeighborType"].map(label_from_type)

valid = (
    (~merged["NeighborAngle"].isna()) &
    (~merged["NEARNEIGHBDIST"].isna()) &
    (merged["NEARNEIGHBDIST"] < DIST_LIMIT)
)

ang_diff = (merged["ORIENTATIONANGLE"] - merged["NeighborAngle"]).abs()
ang_diff = np.where(ang_diff <= 180, ang_diff, 360 - ang_diff)
merged["AngleDiff"] = ang_diff

merged["InteractionType"] = np.where(
    ~valid, "Unknown",
    np.where(ang_diff >= PASS_BY_DEG, "Pass By", "Overtake")
)

def _norm_label(x):
    if x == "Human": return "Human"
    if x == "Robot": return "Robot"
    return "Unknown"

def pair_type_directional(itype, a, b):
    """
    Overtake:   順序付き（Self -> Neighbor） 例: Human-Robot / Robot-Human
    Pass By等:  対称（Human-Robot に丸める）
    """
    a = _norm_label(a)
    b = _norm_label(b)
    if "Unknown" in (a, b):
        return "Unknown"

    if itype == "Overtake":
        # 誰が誰を追い越したかを区別
        return f"{a}-{b}"

    # それ以外は対称扱い
    if a == b:
        return f"{a}-{a}"      # Human-Human / Robot-Robot
    return "Human-Robot"       # 片側に集約

# ← ここが差し替えポイント（InteractionTypeも使う）
merged["PairType"] = [
    pair_type_directional(it, sl, nl)
    for it, sl, nl in zip(merged["InteractionType"], merged["SelfLabel"], merged["NeighborLabel"])
]


# --- イベント開始点検出 ---
merged = merged.sort_values([COL_ID, "SIMSEC"]).reset_index(drop=True)
merged["SelfID"] = merged[COL_ID]  # 後で AgentPair 生成に使用

def to_int_or_nan(x):
    try:    return int(float(x))
    except Exception: return np.nan

merged["NEARNEIGHB_norm"] = merged["NEARNEIGHB"].map(to_int_or_nan)

def mark_starts(g):
    g = g.copy()
    g["prev_type"] = g["InteractionType"].shift(1)
    g["prev_nbr"]  = g["NEARNEIGHB_norm"].shift(1)
    g["prev_t"]    = g["SIMSEC"].shift(1)
    g["is_start"]  = (
        (g["InteractionType"] != g["prev_type"]) |
        (g["NEARNEIGHB_norm"] != g["prev_nbr"]) |
        (g["prev_t"].isna()) |
        ((g["SIMSEC"] - g["prev_t"]) > EVENT_GAP_SEC)
    )
    return g

try:
    merged = merged.groupby(merged[COL_ID], group_keys=False).apply(mark_starts, include_groups=False)
except TypeError:  # pandas < 2.1 互換
    merged = merged.groupby(merged[COL_ID], group_keys=False).apply(mark_starts)

# --- 開始点（自身座標のみ保持）---
def pair_name(a, b):
    try:
        aa, bb = sorted([int(a), int(b)])
        return f"{aa}-{bb}"
    except Exception:
        return f"{a}-{b}"

ev = merged[
    (merged["is_start"]) &
    (~merged["NEARNEIGHB_norm"].isna()) &
    (merged["InteractionType"] != "Unknown") &
    (merged["PairType"] != "Unknown")
].copy()

# ★修正点2：AgentPair は merged 由来の SelfID を参照（df 参照ズレを排除）
ev["AgentPair"] = [pair_name(a, b) for a, b in zip(merged.loc[ev.index, "SelfID"], ev["NEARNEIGHB_norm"])]

ev = (
    ev[["AgentPair","InteractionType","PairType","SIMSEC","SelfX","SelfY","SelfZ"]]
    .rename(columns={"SIMSEC":"StartTime"})
    .sort_values("StartTime")
)

# --- 時刻クラスタリング（代表＝先頭）---
def cluster_times(times, thr):
    times = sorted(t for t in times if pd.notna(t))
    if not times: return []
    clusters, cur = [], [times[0]]
    for t in times[1:]:
        if (t - cur[-1]) <= thr: cur.append(t)
        else: clusters.append(cur); cur = [t]
    clusters.append(cur); return clusters

ev_key = {(r.AgentPair, r.InteractionType, r.PairType, r.StartTime):
          (r.SelfX, r.SelfY, r.SelfZ) for r in ev.itertuples()}

records = []
if not ev.empty:
    for (pair, itype, ptype), g in ev.groupby(["AgentPair","InteractionType","PairType"], sort=False):
        for cl in cluster_times(g["StartTime"].tolist(), CLUSTER_SEC):
            rep_t = cl[0]
            sx, sy, sz = ev_key.get((pair, itype, ptype, rep_t), (np.nan,)*3)
            records.append({
                "AgentPair": pair, "InteractionType": itype, "PairType": ptype,
                "Count": 1, "StartTimes": cl, "RepTime": rep_t,
                "SelfX": sx, "SelfY": sy, "SelfZ": sz
            })

# ★空ガード
final_df = pd.DataFrame(records, columns=[
    "AgentPair","InteractionType","PairType","Count",
    "StartTimes","RepTime","SelfX","SelfY","SelfZ"
])
if not final_df.empty:
    final_df = final_df.sort_values(
        ["AgentPair","InteractionType","PairType","RepTime"]
    ).reset_index(drop=True)

# --- サマリ（空対応 & 列名固定）---
pair_types = ["Human-Human", "Human-Robot", "Robot-Human", "Robot-Robot"]
interaction_types = ["Overtake", "Pass By"]

if final_df.empty:
    summary = pd.DataFrame(
        [(pt, it, 0) for pt in pair_types for it in interaction_types],
        columns=["PairType","InteractionType","Count"]
    )
else:
    summary = (
        final_df.groupby(["PairType","InteractionType"])["Count"]
                .sum().reset_index()
                .set_index(["PairType","InteractionType"])
                .reindex(pd.MultiIndex.from_product([pair_types, interaction_types]), fill_value=0)
                .reset_index()
    )
    summary.columns = ["PairType","InteractionType","Count"]

# === 出力（表示のみ）===
_show(final_df, "=== Events（代表点 / 自身座標のみ） ===", head=15)
_show(summary,  "=== PairType × InteractionType 件数 ===")
_show(ev,       "=== イベント開始点（Raw / 自身座標のみ） ===", head=15)

# === ここでは保存しない。変数に保持する ===
events_final_df  = final_df
events_summary   = summary
events_raw_points = ev

In [ ]:
# === セル3：イベント回数の各種集計（堅牢版） ===
import pandas as pd
import numpy as np

# 表示ユーティリティ（displayが無い環境でも出す）
try:
    from IPython.display import display as _display
except Exception:
    _display = None
def _show(df, title=None):
    if title: print(title)
    if _display:
        _display(df)
    else:
        print(df.to_string(index=False))

# 依存オブジェクト確認（ev / merged / cluster_times / CLUSTER_SEC）
if 'ev' not in globals() or not isinstance(ev, pd.DataFrame):
    raise RuntimeError("ev が見つかりません。セル2（イベント検出）を先に実行してください。")
if 'merged' not in globals() or not isinstance(merged, pd.DataFrame):
    raise RuntimeError("merged が見つかりません。セル2（イベント検出）を先に実行してください。")

# cluster_times / CLUSTER_SEC がなければ最低限を用意
if 'CLUSTER_SEC' not in globals():
    CLUSTER_SEC = 3.0
if 'cluster_times' not in globals():
    def cluster_times(times, thr):
        times = sorted(t for t in times if pd.notna(t))
        if not times: return []
        clusters, cur = [], [times[0]]
        for t in times[1:]:
            if (t - cur[-1]) <= thr: cur.append(t)
            else: clusters.append(cur); cur = [t]
        clusters.append(cur); return clusters

# ---- 便利関数 ----
def ped_to_agent_cat(pedtype):
    """100/200→Human、300→Robot、それ以外→Unknown"""
    try:
        v = int(float(pedtype))
    except Exception:
        return "Unknown"
    if v in (100, 200): return "Human"
    if v == 300:        return "Robot"
    return "Unknown"

# 列名ゆらぎ検出（dfが必要）
def norm(s: str) -> str:
    import unicodedata as ud
    s = ud.normalize("NFKC", str(s)).strip().lower()
    s = s.replace("$","").replace("：",":").replace("（","(").replace("）",")")
    for ch in [" ", "\t", "-", "/", "\\", ":"]:
        s = s.replace(ch, "_")
    while "__" in s: s = s.replace("__","_")
    return s
def find_col(df_: pd.DataFrame, candidates) -> str:
    nmap = {c: norm(c) for c in df_.columns}
    wants = {norm(c) for c in candidates}
    for col, coln in nmap.items():
        if coln in wants:
            return col
    return ""

if 'df' not in globals() or not isinstance(df, pd.DataFrame):
    raise RuntimeError("df が見つかりません。読み込みセルを先に実行してください。")

# --- Self 情報の復元（ev の index は merged の index継承前提） ---
_ev_work = ev.copy()
_ev_work["SelfID"]      = merged.loc[_ev_work.index, "SelfID"].values
_ev_work["SelfPEDTYPE"] = merged.loc[_ev_work.index, "PEDTYPE"].values
_ev_work["SelfCat"]     = _ev_work["SelfPEDTYPE"].map(ped_to_agent_cat)

# --- 代表イベントへ再クラスタ（クラスタ先頭を代表） ---
records = []
for (pair, itype, ptype), g in _ev_work.groupby(["AgentPair","InteractionType","PairType"], sort=False):
    times = sorted(g["StartTime"].tolist())
    clusters = cluster_times(times, CLUSTER_SEC)
    for cl in clusters:
        rep_t = cl[0]
        gi = g.loc[g["StartTime"] == rep_t].iloc[0]  # 同時刻複数でも先頭
        records.append({
            "AgentPair": pair,
            "InteractionType": itype,
            "PairType": ptype,
            "RepTime": rep_t,
            "SelfID": gi["SelfID"],
            "SelfPEDTYPE": gi["SelfPEDTYPE"],
            "SelfCat": ped_to_agent_cat(gi["SelfPEDTYPE"])
        })

events = pd.DataFrame.from_records(records)
if events.empty:
    events = pd.DataFrame(columns=[
        "AgentPair","InteractionType","PairType","RepTime",
        "SelfID","SelfPEDTYPE","SelfCat"
    ])

# ===== ① 全体のイベント回数 =====
overall_events_count = len(events)
print(f"【全体のイベント回数】 {overall_events_count}")

# ===== ② PEDTYPE別（Self）イベント回数 =====
cat_counts = (
    events.query("SelfCat in ['Human','Robot']")
          .groupby("SelfCat", as_index=False)
          .size()
          .rename(columns={"size":"Count"})
)
cat_counts["SelfCat"] = pd.Categorical(cat_counts["SelfCat"], ["Human","Robot"])
cat_counts = cat_counts.sort_values("SelfCat").reset_index(drop=True)
print("\n【PEDTYPE別（Self）イベント回数】")
_show(cat_counts)

# ===== ③ PEDTYPE別 × 1800秒ビンのイベント回数（ピボット） =====
if not events.empty:
    events_time = events.copy()
    events_time["TimeBinStart"] = (events_time["RepTime"] // 1800 * 1800).astype(float)
    time_cat_counts = (
        events_time.query("SelfCat in ['Human','Robot']")
                   .groupby(["TimeBinStart","SelfCat"], as_index=False)
                   .size()
                   .rename(columns={"size":"Count"})
    )
    time_cat_pivot = (
        time_cat_counts.pivot(index="TimeBinStart", columns="SelfCat", values="Count")
                       .fillna(0).astype(int).sort_index()
    )
else:
    time_cat_counts = pd.DataFrame(columns=["TimeBinStart","SelfCat","Count"])
    time_cat_pivot  = pd.DataFrame(columns=["Human","Robot"]).astype(int)
print("\n【PEDTYPE別 × 1800秒ビンごとのイベント回数】（ピボット）")
_show(time_cat_pivot)

# ===== ④ PEDTYPE別 × STAROUTDECNO ごとのイベント回数 =====
COL_T    = find_col(df, ["SIMSEC","TIME","TIMESTAMP","SEC"])
COL_ID   = find_col(df, ["$PEDESTRIAN:NO","PEDESTRIAN:NO","PEDESTRIAN_NO","AGENT_ID","ID","AgentID"])
COL_STAR = find_col(df, ["STAROUTDECNO","STAR_OUT_DEC_NO","STAROUT_DEC_NO","STAROUTDEC_NO","STAROUT_DECNO","STAROUTDEC"])

star_counts = pd.DataFrame(columns=["SelfCat","STAROUTDECNO","Count"])

if COL_T and COL_ID and COL_STAR and not events.empty:
    tmp = df[[COL_ID, COL_T, COL_STAR]].copy()
    tmp = tmp.rename(columns={COL_ID:"ID", COL_T:"SIMSEC", COL_STAR:"STAROUTDECNO"})
    tmp["ID"]           = pd.to_numeric(tmp["ID"], errors="coerce")
    tmp["SIMSEC"]       = pd.to_numeric(tmp["SIMSEC"], errors="coerce")
    tmp["STAROUTDECNO"] = pd.to_numeric(tmp["STAROUTDECNO"], errors="coerce")

    star_results = []
    for aid, g in tmp.dropna(subset=["ID","SIMSEC"]).sort_values(["ID","SIMSEC"]).groupby("ID", sort=False):
        g = g[["SIMSEC","STAROUTDECNO"]].copy().reset_index(drop=True)
        e_sub = events.loc[events["SelfID"] == aid, ["RepTime","SelfCat"]].copy()
        if e_sub.empty:
            continue
        e_sub = e_sub.sort_values("RepTime").reset_index(drop=True)
        e_sub_ren = e_sub.rename(columns={"RepTime":"SIMSEC"})
        merged_star = pd.merge_asof(
            e_sub_ren.sort_values("SIMSEC"),
            g, on="SIMSEC", direction="backward", tolerance=1800.0  # 直前の値を採用（許容広め）
        )
        merged_star = merged_star.rename(columns={"SIMSEC":"RepTime"})
        merged_star["SelfID"] = aid
        star_results.append(merged_star)

    if star_results:
        star_ev = pd.concat(star_results, ignore_index=True)
        star_counts = (
            star_ev.query("SelfCat in ['Human','Robot'] and STAROUTDECNO.notna()")
                   .groupby(["SelfCat","STAROUTDECNO"], as_index=False)
                   .size().rename(columns={"size":"Count"})
                   .sort_values(["SelfCat","STAROUTDECNO"]).reset_index(drop=True)
        )
else:
    if not COL_STAR:
        print("\n⚠️ 注意: STAROUTDECNO 列が見つかりませんでした。④の集計はスキップします。")
    elif events.empty:
        print("\n⚠️ 注意: イベントが空のため、④の集計はスキップします。")

print("\n【PEDTYPE別 × STAROUTDECNO ごとのイベント回数】")
_show(star_counts)

# === ここでは保存しない：変数として保持 ===
events_overall_df        = pd.DataFrame({"OverallEvents":[overall_events_count]})
events_counts_by_type    = cat_counts
events_counts_time_counts= time_cat_counts
events_counts_time_pivot = time_cat_pivot
events_counts_by_star    = star_counts

In [ ]:
# ===== セル：Floor別イベント数（表のみ）＆セル別カウント行列（全時間→時間×階層）【画像なし版】 =====
import pandas as pd
import numpy as np
from pathlib import Path

# --- display フォールバック ---
try:
    from IPython.display import display as _display
except Exception:
    _display = None
def _show(df, title=None, head=None):
    if title: print(title)
    if df is None:
        print("(no data)")
        return
    if head is not None and hasattr(df, "head"):
        df = df.head(head)
    if _display:
        _display(df)
    else:
        try:
            print(df.to_string(index=False))
        except Exception:
            print(df)

# --- 入力（セル2の出力を使用） ---
if "final_df" in globals() and isinstance(final_df, pd.DataFrame) and not final_df.empty:
    src = final_df.copy()                          # 代表イベント（クラスタ済み）
    TIME_COL = "RepTime"
elif "ev" in globals() and isinstance(ev, pd.DataFrame) and not ev.empty:
    src = ev.copy().rename(columns={"StartTime":"RepTime"})  # 開始点
    TIME_COL = "RepTime"
else:
    raise RuntimeError("イベントデータが見つかりません（final_df または ev）。先にセル2を実行してください。")

# --- パラメータ（必要なら変更） ---
CELL_SIZE_M     = 1.0     # 1セルのグリッドサイズ[m]
FLOOR_DECIMALS  = 1       # SelfZの丸め桁（1=0.1m単位, 0=1m単位 など）
TIME_BIN_SEC    = 1800.0  # 時間ビン幅[秒]

# --- 必須列チェック ---
need = {"SelfX","SelfY","SelfZ", TIME_COL}
miss = [c for c in need if c not in src.columns]
if miss:
    raise RuntimeError(f"必要列が不足しています: {miss}")

# --- 前処理 ---
data = src.dropna(subset=["SelfX","SelfY","SelfZ", TIME_COL]).copy()
if data.empty:
    print("⚠️ 有効なイベント行（SelfX/Y/Z と時間）がありません。空の出力を保持します。")
    # 空でも後続セルが落ちないように空の器を用意
    floor_counts = pd.DataFrame(columns=["FloorZ","Count"])
    x_edges = y_edges = np.array([0,1], dtype=float)
    # ヒートマップに相当するセル別カウントの行列（CSV化予定）の格納先
    heat_alltime_counts = []  # [{csv_name, counts_df}]
    heat_time_counts    = []  # [{csv_name, counts_df}]
else:
    # 数値化（保険）
    for c in ["SelfX","SelfY","SelfZ", TIME_COL]:
        data[c] = pd.to_numeric(data[c], errors="coerce")
    data = data.dropna(subset=["SelfX","SelfY","SelfZ", TIME_COL]).copy()

    data["FloorZ"] = data["SelfZ"].round(FLOOR_DECIMALS)
    data["TimeBinStart"] = (np.floor_divide(data[TIME_COL].astype(float), TIME_BIN_SEC) * TIME_BIN_SEC).astype(float)

    # --- グリッド範囲（全図共通） ---
    def get_bounds(x, y, pad_ratio=0.02, min_pad=0.5):
        x = np.asarray(x, dtype=float); y = np.asarray(y, dtype=float)
        xmin, xmax = float(np.nanmin(x)), float(np.nanmax(x))
        ymin, ymax = float(np.nanmin(y)), float(np.nanmax(y))
        # 同一点ばかりのときに幅ゼロにならないようにパディング
        xrange = max(xmax - xmin, min_pad)
        yrange = max(ymax - ymin, min_pad)
        xpad = max(xrange * pad_ratio, CELL_SIZE_M) * 0.5
        ypad = max(yrange * pad_ratio, CELL_SIZE_M) * 0.5
        return (xmin - xpad, xmax + xpad, ymin - ypad, ymax + ypad)

    xmin, xmax, ymin, ymax = get_bounds(data["SelfX"].values, data["SelfY"].values)

    # 端を含むエッジ列（数値誤差対策）
    eps = 1e-9
    x_edges = np.arange(xmin - eps, xmax + CELL_SIZE_M + eps, CELL_SIZE_M)
    y_edges = np.arange(ymin - eps, ymax + CELL_SIZE_M + eps, CELL_SIZE_M)
    # bins は最低 2 本必要
    if len(x_edges) < 2: x_edges = np.array([xmin - 0.5, xmin + 0.5])
    if len(y_edges) < 2: y_edges = np.array([ymin - 0.5, ymin + 0.5])

    # ===== 1) Event Count by Floor（表のみ表示・保持） =====
    floor_counts = (
        data.groupby("FloorZ", as_index=False)
            .size()
            .rename(columns={"size": "Count"})
            .sort_values("FloorZ")
            .reset_index(drop=True)
    )
    print("【Event Count by Floor（階層ごとのイベント数）】")
    _show(floor_counts)

    # ===== 2) 全時間 × 各階層：セル別カウント行列を作成・保持 =====
    heat_alltime_counts = []  # [{csv_name, counts_df}]
    for fz, g in data.groupby("FloorZ", sort=True):
        df = g.dropna(subset=["SelfX","SelfY"])
        if df.empty:
            continue
        H, xe, ye = np.histogram2d(df["SelfX"], df["SelfY"], bins=[x_edges, y_edges])
        counts_df = pd.DataFrame(
            H,
            index=pd.IntervalIndex.from_breaks(xe).astype(str),
            columns=pd.IntervalIndex.from_breaks(ye).astype(str),
        )
        heat_alltime_counts.append({
            "csv_name": f"heatmap_alltime_floor_{str(fz).replace('.','p')}_counts.csv",
            "counts_df": counts_df
        })

    # ===== 3) 時間ごと × 各階層：セル別カウント行列を作成・保持 =====
    heat_time_counts = []  # [{csv_name, counts_df}]
    for tb, gtb in data.groupby("TimeBinStart", sort=True):
        tlabel = int(tb) if pd.notna(tb) else -1
        for fz, gf in gtb.groupby("FloorZ", sort=True):
            df = gf.dropna(subset=["SelfX","SelfY"])
            if df.empty:
                continue
            H, xe, ye = np.histogram2d(df["SelfX"], df["SelfY"], bins=[x_edges, y_edges])
            counts_df = pd.DataFrame(
                H,
                index=pd.IntervalIndex.from_breaks(xe).astype(str),
                columns=pd.IntervalIndex.from_breaks(ye).astype(str),
            )
            heat_time_counts.append({
                "csv_name": f"heatmap_time_{tlabel}_floor_{str(fz).replace('.','p')}_counts.csv",
                "counts_df": counts_df
            })

# === このセルでは保存しない ===
# 以下の変数を“最後のセル”で保存に使います：
# floor_counts, x_edges, y_edges, CELL_SIZE_M, TIME_BIN_SEC,
# heat_alltime_counts (リスト), heat_time_counts (リスト)


In [ ]:
"""
# === まとめて保存（最後のセル・整理版） ===
from pathlib import Path
import pandas as pd
import numpy as np
import json
import numbers
import datetime as dt

# ---------- 出力フォルダ ----------
OUT_MAIN      = Path("outputs")                # 旅行時間・分散など
OUT_STATS     = OUT_MAIN / "stats"
OUT_INTERM    = OUT_MAIN / "intermediate"
OUT_EVENTS    = Path("outputs_events")         # イベント回数・イベント結果
OUT_HEAT      = Path("heatmaps_min")           # 画像なしヒートマップ（セル別カウントCSV）

for p in [OUT_MAIN, OUT_STATS, OUT_INTERM, OUT_EVENTS, OUT_HEAT]:
    p.mkdir(parents=True, exist_ok=True)

# ---------- ヘルパ ----------
def _df_ok(name): return (name in globals()) and isinstance(globals()[name], pd.DataFrame)
def _num_ok(x):   return isinstance(x, numbers.Number)

def _save_csv(df, path, index=False):
    df.to_csv(path, index=index, encoding="utf-8-sig")
    # print(f"[保存] {path}")  # 必要ならコメント解除

# ==============================================================
# 1) 旅行時間系（第一/第二処理の成果）
# ==============================================================
if _df_ok("result"):
    _save_csv(result, OUT_MAIN / "travel_times_by_ped.csv")
else:
    print("[注意] result なし：旅行時間（歩行者別）をスキップ。")

if _df_ok("by_type"):
    _save_csv(by_type, OUT_MAIN / "avg_travel_time_by_agent_type.csv")
if _df_ok("by_bin"):
    _save_csv(by_bin, OUT_MAIN / "avg_travel_time_by_agent_and_timebin.csv")
if _df_ok("result_with_type"):
    _save_csv(result_with_type, OUT_MAIN / "travel_times_with_agent_type.csv")
if _df_ok("df_sorted"):
    _save_csv(df_sorted, OUT_INTERM / "df_sorted.csv")

# --- AGENT_TYPE × STAROUTDECNO ---
if _df_ok("avg_travel_by_agent_star"):
    _save_csv(avg_travel_by_agent_star, OUT_MAIN / "avg_travel_time_by_agent_star.csv")
if _df_ok("result_with_meta"):
    _save_csv(result_with_meta, OUT_MAIN / "travel_time_with_agent_star_meta.csv")

# --- 元 df を .parsed.csv で（PATH があれば同ディレクトリ） ---
if _df_ok("df"):
    src = None
    try:
        if "PATH" in globals() and PATH:
            src = Path(PATH)
    except Exception:
        src = None

    out_path_df = (src.with_suffix(".parsed.csv") if src else OUT_MAIN / "df.parsed.csv")
    out_path_df.parent.mkdir(parents=True, exist_ok=True)
    _save_csv(df, out_path_df)

# --- スカラー（overall/total） ---
if "overall" in globals() and _num_ok(overall):
    (OUT_STATS / "overall_avg_travel_time.txt").write_text(str(overall), encoding="utf-8")
    (OUT_STATS / "overall_avg_travel_time.json").write_text(
        json.dumps({"overall_avg_travel_time": float(overall)}), encoding="utf-8"
    )
else:
    print("[注意] overall なし/数値でない：全体平均TRAVEL_TIMEをスキップ。")

if "total" in globals() and _num_ok(total):
    (OUT_STATS / "total_travel_time_sum.txt").write_text(str(total), encoding="utf-8")
    (OUT_STATS / "total_travel_time_sum.json").write_text(
        json.dumps({"total_travel_time_sum": float(total)}), encoding="utf-8"
    )
else:
    print("[注意] total なし/数値でない：TRAVEL_TIME総和をスキップ。")

# ==============================================================
# 2) 速度分散の集計（overall_df / by_type / by_type_time / by_type_star）
# ==============================================================
if _df_ok("overall_df"):
    _save_csv(overall_df, OUT_MAIN / "mean_var_overall.csv")
else:
    print("[注意] overall_df なし：分散の全体平均をスキップ。")
if _df_ok("by_type"):
    _save_csv(by_type, OUT_MAIN / "mean_var_by_type.csv")
if _df_ok("by_type_time"):
    _save_csv(by_type_time, OUT_MAIN / "mean_var_by_type_timebin.csv")
if _df_ok("by_type_star"):
    _save_csv(by_type_star, OUT_MAIN / "mean_var_by_type_star.csv")

# メタ（速度分散）
mv_meta = {}
if "BIN" in globals() and _num_ok(BIN): mv_meta["BIN_seconds"] = int(BIN)
if "var_col" in globals():              mv_meta["var_col"] = var_col
if "time_col" in globals():             mv_meta["time_col"] = time_col
mv_meta["generated_at"] = dt.datetime.now().isoformat()
(OUT_STATS / "mean_var_meta.json").write_text(json.dumps(mv_meta, ensure_ascii=False, indent=2), encoding="utf-8")

# ==============================================================
# 3) 速度分散：第二処理（ラベル件数）
# ==============================================================
if _df_ok("level_counts"):
    _save_csv(level_counts, OUT_MAIN / "expvelvar_level_summary.csv")
else:
    print("[注意] level_counts なし：レベル総数をスキップ。")
if _df_ok("level_by_ped"):
    _save_csv(level_by_ped, OUT_MAIN / "expvelvar_level_by_ped.csv")

# ==============================================================
# 4) イベント回数（セル3）
# ==============================================================
if _df_ok("events_overall_df"):
    _save_csv(events_overall_df, OUT_EVENTS / "overall_event_count.csv")
else:
    print("[注意] events_overall_df なし：イベント総数CSVをスキップ。")

if _df_ok("events_counts_by_type"):
    _save_csv(events_counts_by_type, OUT_EVENTS / "event_counts_by_PEDTYPE.csv")
if _df_ok("events_counts_time_counts"):
    _save_csv(events_counts_time_counts, OUT_EVENTS / "event_counts_by_PEDTYPE_timebin1800.csv")
if _df_ok("events_counts_time_pivot"):
    # ピボットは index 付きで保存（行ラベル＝TimeBinStart）
    events_counts_time_pivot.to_csv(OUT_EVENTS / "event_counts_by_PEDTYPE_timebin1800_pivot.csv",
                                    encoding="utf-8-sig")
if _df_ok("events_counts_by_star"):
    _save_csv(events_counts_by_star, OUT_EVENTS / "event_counts_by_PEDTYPE_STAROUTDECNO.csv")

# メタ（イベント回数セル3）
ev_meta = {"time_bin_seconds": 1800}
if "CLUSTER_SEC" in globals() and _num_ok(CLUSTER_SEC): ev_meta["CLUSTER_SEC"] = float(CLUSTER_SEC)
(OUT_EVENTS / "events_meta.json").write_text(json.dumps(ev_meta, ensure_ascii=False, indent=2), encoding="utf-8")

# ==============================================================
# 5) イベント検出の最終/中間（final_df/summary/raw）
# ==============================================================
if _df_ok("events_final_df"):
    _save_csv(events_final_df, OUT_EVENTS / "final_events_self_only.csv")
else:
    print("[注意] events_final_df なし：final をスキップ。")
if _df_ok("events_summary"):
    _save_csv(events_summary, OUT_EVENTS / "pairtype_summary.csv")
if _df_ok("events_raw_points"):
    _save_csv(events_raw_points, OUT_EVENTS / "event_points_self_only.csv")

# 閾値メタ（イベント検出セル）
ev_thresh = {}
for k in ["TOL_NEIGHBOR_SEC","EVENT_GAP_SEC","CLUSTER_SEC","DIST_LIMIT","PASS_BY_DEG"]:
    if k in globals():
        ev_thresh[k] = globals()[k]
(OUT_EVENTS / "events_thresholds.json").write_text(json.dumps(ev_thresh, ensure_ascii=False, indent=2), encoding="utf-8")

# ==============================================================
# 6) 画像なしヒートマップ（セル別カウントCSVのみ）
# ==============================================================
if _df_ok("floor_counts") and not floor_counts.empty:
    _save_csv(floor_counts, OUT_HEAT / "event_count_by_floor.csv")
else:
    print("[注意] floor_counts なし/空：階層別イベント数CSVをスキップ。")

if "heat_alltime_counts" in globals():
    for spec in heat_alltime_counts:
        _save_csv(spec["counts_df"], OUT_HEAT / spec["csv_name"])
else:
    print("[注意] heat_alltime_counts 未定義：全時間カウントCSVをスキップ。")

if "heat_time_counts" in globals():
    for spec in heat_time_counts:
        _save_csv(spec["counts_df"], OUT_HEAT / spec["csv_name"])
else:
    print("[注意] heat_time_counts 未定義：時間×階層カウントCSVをスキップ。")

# メタ（グリッド）
grid_meta = {}
if "CELL_SIZE_M"  in globals(): grid_meta["CELL_SIZE_M"]  = float(CELL_SIZE_M)
if "TIME_BIN_SEC" in globals(): grid_meta["TIME_BIN_SEC"] = float(TIME_BIN_SEC)
if "x_edges"      in globals(): grid_meta["X_bins"]       = int(len(x_edges)-1)
if "y_edges"      in globals(): grid_meta["Y_bins"]       = int(len(y_edges)-1)
(OUT_HEAT / "grid_meta.json").write_text(json.dumps(grid_meta, ensure_ascii=False, indent=2), encoding="utf-8")

# ==============================================================
# 7) 終了メッセージ
# ==============================================================
print("\n✅ 保存完了")
print(" -", OUT_MAIN.resolve())
print(" -", OUT_EVENTS.resolve())
print(" -", OUT_HEAT.resolve())
"""


In [ ]:
# === 最後のセル：指定シートのみを1つのExcelに保存 ===
from pathlib import Path
import pandas as pd
import json, numbers, datetime as dt
import re

# 出力先とファイル名
OUT_DIR = Path("outputs_all_in_one_1111デフォルト")
OUT_DIR.mkdir(parents=True, exist_ok=True)
ts = dt.datetime.now().strftime("%Y%m%d-%H%M%S")
xlsx_path = OUT_DIR / f"analysis_outputs_{ts}.xlsx"

def _df_ok(name):
    return (name in globals()) and isinstance(globals()[name], pd.DataFrame)

def _num_ok(x):
    return isinstance(x, numbers.Number)

def _to_df(obj, key_name="key", val_name="value"):
    """dict/数値などを2列DataFrameへ"""
    if isinstance(obj, dict):
        return pd.DataFrame([(k, obj[k]) for k in obj], columns=[key_name, val_name])
    if _num_ok(obj):
        return pd.DataFrame({val_name:[obj]})
    return pd.DataFrame({val_name:[json.dumps(obj, ensure_ascii=False)]})

# シート名を安全化（31文字制限・禁止文字・重複対策）
def _safe_sheet_name(name, used):
    # 禁止文字: : \ / ? * [ ]
    name = re.sub(r'[:\\/\?\*\[\]]', '_', str(name))
    name = name.strip().strip("'")  # 先頭/末尾の ' は避ける
    if not name:
        name = "Sheet"
    # 既に使われていないかチェック
    base = name[:31]
    if base not in used:
        used.add(base)
        return base
    # 衝突したら連番付与
    for i in range(1, 10000):
        suffix = f"_{i}"
        cand = (base[:31-len(suffix)]) + suffix
        if cand not in used:
            used.add(cand)
            return cand
    # 異常系：最後の手段
    fallback = f"Sheet_{len(used)+1}"
    used.add(fallback)
    return fallback

# ここに「シート名 → DataFrame」を順次積む
sheets = []

# ===================== ここから出力対象のみ =====================
# ★ events_raw_points → "events_raw_points"
if _df_ok("events_raw_points"):
    sheets.append(("events_raw_points", events_raw_points))

# ★ events_pairtype_summary → "events_pairtype_summary"
#    （変数名は events_summary を想定）
if _df_ok("events_summary"):
    sheets.append(("events_pairtype_summary", events_summary))

# ★ events_by_PEDTYPE_STAROUTDECNO → "events_by_PEDTYPE_STAROUTDECNO"
#    （変数名は events_counts_by_star を想定）
if _df_ok("events_counts_by_star"):
    sheets.append(("events_by_PEDTYPE_STAROUTDECNO", events_counts_by_star))

# ★ events_by_PEDTYPE → "events_by_PEDTYPE"
#    （変数名は events_counts_by_type を想定）
if _df_ok("events_counts_by_type"):
    sheets.append(("events_by_PEDTYPE", events_counts_by_type))
# ===================== ここまで =====================

# ---------------- 以下はすべて無効化 ----------------
# if _df_ok("result"):                sheets.append(("travel_times_by_ped", result))
# if _df_ok("by_bin"):                sheets.append(("avg_travel_time_by_agent_timebin", by_bin))
# if _df_ok("result_with_type"):      sheets.append(("travel_times_with_agent_type", result_with_type))
# if _df_ok("df_sorted"):             sheets.append(("df_sorted", df_sorted))
# if _df_ok("avg_travel_by_agent_star"): sheets.append(("avg_travel_time_by_agent_star", avg_travel_by_agent_star))
# if _df_ok("result_with_meta"):      sheets.append(("agent_star_meta_join", result_with_meta))
# if "overall" in globals() and _num_ok(overall): sheets.append(("overall_avg_travel_time", _to_df(overall, val_name="overall")))
# if "total"   in globals() and _num_ok(total):   sheets.append(("total_travel_time_sum", _to_df(total,   val_name="total")))
# if _df_ok("by_type"):
#     cols = set(map(str, by_type.columns))
#     if "TRAVEL_TIME" in cols:
#         sheets.append(("avg_travel_time_by_agent_type", by_type))
#     if "MEAN_VAR" in cols:
#         sheets.append(("mean_var_by_type", by_type))
# if _df_ok("episodes"):      sheets.append(("expvelvar_episodes", episodes))
# if _df_ok("count_table"):   sheets.append(("expvelvar_counts_by_ped", count_table))
# if _df_ok("abnormal"):      sheets.append(("expvelvar_abnormal_only", abnormal))
# if _df_ok("overall_df"):    sheets.append(("mean_var_overall", overall_df))
# if _df_ok("by_type_time"):  sheets.append(("mean_var_by_type_timebin", by_type_time))
# if _df_ok("by_type_star"):  sheets.append(("mean_var_by_type_star", by_type_star))
# mv_meta = {}
# if "BIN" in globals() and _num_ok(BIN): mv_meta["BIN_seconds"] = int(BIN)
# if "var_col" in globals():               mv_meta["var_col"] = var_col
# if "time_col" in globals():              mv_meta["time_col"] = time_col
# mv_meta["generated_at"] = dt.datetime.now().isoformat()
# sheets.append(("mean_var_meta", _to_df(mv_meta)))
if _df_ok("events_overall_df"):          sheets.append(("events_overall_count", events_overall_df))
# if _df_ok("events_counts_time_counts"):  sheets.append(("events_by_PEDTYPE_timebin1800", events_counts_time_counts))
# if _df_ok("events_counts_time_pivot"):   sheets.append(("events_by_PEDTYPE_timebin_pivot", events_counts_time_pivot))
# ev_meta = {"time_bin_seconds": 1800}
# if "CLUSTER_SEC" in globals() and _num_ok(CLUSTER_SEC): ev_meta["CLUSTER_SEC"] = float(CLUSTER_SEC)
# sheets.append(("events_meta", _to_df(ev_meta)))
# if _df_ok("events_final_df"):   sheets.append(("events_final_self_only", events_final_df))
# ev_thr = {}
# for k in ["TOL_NEIGHBOR_SEC","EVENT_GAP_SEC","CLUSTER_SEC","DIST_LIMIT","PASS_BY_DEG"]:
#     if k in globals(): ev_thr[k] = globals()[k]
# sheets.append(("events_thresholds", _to_df(ev_thr)))
# 画像・ヒートマップ系、raw_df などもすべて無効化
# ----------------------------------------------------

# ===== 書き込み =====
used_names = set()

def _open_writer(path):
    # 1) xlsxwriter → 2) openpyxl → 3) エンジン指定なし（pandasの既定）
    for eng in ("xlsxwriter", "openpyxl", None):
        try:
            return pd.ExcelWriter(path, engine=eng) if eng else pd.ExcelWriter(path)
        except ModuleNotFoundError:
            continue
    raise ModuleNotFoundError("Neither 'xlsxwriter' nor 'openpyxl' is installed.")

with _open_writer(xlsx_path) as w:
    for name, df in sheets:
        try:
            sheet = _safe_sheet_name(name, used_names)
            df.to_excel(w, sheet_name=sheet, index=False)
        except Exception as e:
            print(f"[警告] シート {name} の書き込みに失敗: {e}")
